Let's test the vector field to make sure it's working...

In [1]:
from operator import index
import os
from pathlib import Path
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from openai import AzureOpenAI  # If you're using Azure OpenAI for embeddings
from dotenv import load_dotenv
from azure.search.documents.models import VectorizedQuery


# Get root directory path
root_dir = Path().absolute().parent
env_path = root_dir / '.env'

# Load .env from root
load_dotenv(dotenv_path=env_path)
print(f"Loaded .env from {env_path}")
# Access variables
api_key = os.getenv('AZURE_OPENAI_API_KEY')
debug = os.getenv('DEBUG')
more_research = os.getenv('MORE_RESEARCH')
index_name = os.getenv('AZURE_SEARCH_INDEX')


print(f"API Key: {  api_key[:4] + '*' * 28 + api_key[-4:] }")
print(f"Index Name: {index_name}")


Loaded .env from c:\Users\rickcau\source\repos\vendor-contracts-gen-ai\.env
API Key: 8PVz****************************Isv1
Index Name: rdc-contracts-v1


TBD

In [ ]:
search_endpoint = os.getenv('AZURE_SEARCH_ENDPOINT')
index_name = os.getenv('INDEX_NAME')
search_key = os.getenv('AZURE_SEARCH_KEY')
index_name = os.getenv('AZURE_SEARCH_INDEX')

aoai_client = AzureOpenAI(
        azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
        api_key=os.getenv("AZURE_OPENAI_KEY"),  
        api_version="2023-05-15"
        )

def generate_embeddings(text, model="text-embedding-ada-002"): # model = "deployment_name"
    return aoai_client.embeddings.create(input = [text], model=model).data[0].embedding

def filter_search_results(results):
    filtered_results = []
    for result in results:
        base_score = result.get('@search.score', 0)
        reranker_score = result.get('@search.reranker_score', 0)
        highlights = result.get('@search.highlights')
        
        # For vendor name searches, require either:
        # 1. Text highlights present (direct matches) OR
        # 2. Higher minimum scores for non-highlighted results
        if highlights:
            # If we have highlights, use lower thresholds
            if base_score >= 0.01 and reranker_score >= 1.0:
                filtered_results.append(result)
        else:
            # No highlights - require higher scores to compensate
            if base_score >= 0.05 and reranker_score >= 3.0:
                filtered_results.append(result)
    
    return filtered_results if filtered_results else None

# Test vector search
search_client = SearchClient(endpoint=search_endpoint , index_name=index_name, credential=AzureKeyCredential(search_key))

query = "Fabrikam"
query_vector = generate_embeddings(query)

top = 3
vector_query = VectorizedQuery(vector=query_vector, k_nearest_neighbors=top, fields="vendorNameVector", kind="vector")
print(f"\nFirst 1: using: {query}")
# results = search_client.search(
#     search_text=query,  
#     vector_queries=[vector_query],
#     select=["id", "contractId", "vendorName"]
# )

results = search_client.search(
    search_text=query,
    vector_queries=[vector_query],
    select=["id", "contractId", "vendorName"],
    highlight_fields="vendorName",
    query_type="semantic",  # Enable semantic search
    semantic_configuration_name="default",  # You'll need to set this up
)

filtered_results=filter_search_results(results) # removed the items that are not good enough

for result in filtered_results:
    print(result)

query = "What can you tell me about the vendor agreement between Fabrikam Services and Contoso Elite?"
# Option 1: Pure Text Search with Weights
print(f"\nPure Text Search with Weights, using: {query} ")
results = search_client.search(
    search_text= query,
    select=["id", "contractId", "vendorName"], 
    highlight_fields="vendorName",
)

for result in results:
    print(result)
    
# Option 2: Hybrid Search with Proper Query
query = "X"
print(f"\nHybrid Search with Proper Query, using: {query} ")

query_vector = generate_embeddings(query)  

vector_query = VectorizedQuery(vector=query_vector, k_nearest_neighbors=3, fields="vendorNameVector", kind="vector")

results = search_client.search(
    search_text=query,
    vector_queries=[vector_query],
    select=["id", "contractId", "vendorName"],
    highlight_fields="vendorName",
    query_type="semantic",  # Enable semantic search
    semantic_configuration_name="default",  # You'll need to set this up
)

filtered_results=filter_search_results(results) # removed the items that are not good enough

if filtered_results:  # if there are any results
    for result in filtered_results:
        print(result)
else:
    print("No results found")




First 1: using: Fabrikam
{'id': '1fb91887f558ee99f577956e7f6701df', 'vendorName': 'Fabrikam Services', 'contractId': '5004432', '@search.score': 0.03306011110544205, '@search.reranker_score': 2.5451736450195312, '@search.highlights': {'vendorName': ['<em>Fabrikam</em> Services']}, '@search.captions': None}
{'id': 'f310326ed56592ebcef7c87b142efa32', 'vendorName': 'Fabrikam Services', 'contractId': '5004432-A1', '@search.score': 0.03306011110544205, '@search.reranker_score': 2.4604554176330566, '@search.highlights': {'vendorName': ['<em>Fabrikam</em> Services']}, '@search.captions': None}

Pure Text Search with Weights, using: What can you tell me about the vendor agreement between Fabrikam Services and Contoso Elite? 
{'vendorName': 'Fabrikam Services', 'contractId': '5004432', 'id': '1fb91887f558ee99f577956e7f6701df', '@search.score': 5.528987, '@search.reranker_score': None, '@search.highlights': {'vendorName': ['<em>Fabrikam</em> <em>Services</em>']}, '@search.captions': None}
{'ven

sdfsdf

In [24]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery
from azure.core.credentials import AzureKeyCredential
from openai import AzureOpenAI
import os

# Setup clients
search_endpoint = os.getenv('AZURE_SEARCH_ENDPOINT')
index_name = os.getenv('AZURE_SEARCH_INDEX')
search_key = os.getenv('AZURE_SEARCH_KEY')

aoai_client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
    api_key=os.getenv("AZURE_OPENAI_KEY"),  
    api_version="2023-05-15"
)

search_client = SearchClient(
    endpoint=search_endpoint,
    index_name=index_name,
    credential=AzureKeyCredential(search_key)
)

def generate_embeddings(text, model="text-embedding-ada-002"):
    return aoai_client.embeddings.create(input=[text], model=model).data[0].embedding

def filter_search_results(results):
    filtered_results = []
    for result in results:
        base_score = result.get('@search.score', 0)
        reranker_score = result.get('@search.reranker_score', 0)
        highlights = result.get('@search.highlights')
        
        # For vendor name searches, require either:
        # 1. Text highlights present (direct matches) OR
        # 2. Higher minimum scores for non-highlighted results
        if highlights:
            # If we have highlights, use lower thresholds
            if base_score >= 0.01 and reranker_score >= 1.0:
                filtered_results.append(result)
        else:
            # No highlights - require higher scores to compensate
            if base_score >= 0.05 and reranker_score >= 3.0:
                filtered_results.append(result)
    
    return filtered_results if filtered_results else None

def compare_searches(query: str):
    # Generate vector for hybrid search
    query_vector = generate_embeddings(query)
    
    vector_query = VectorizedQuery(
        vector=query_vector,
        k_nearest_neighbors=3,
        fields="vendorNameVector",
        kind="vector"
    )

    # Pure text search with all searchable fields
    text_results = search_client.search(
        search_text=query,
        select=["id", "contractId", "vendorName", "content"],
        search_fields=["vendorName", "clientName","contractTitle","sourceFileName","content"],
        top=3
    )

    # Hybrid search with all searchable fields
    hybrid_results = search_client.search(
        search_text=query,
        vector_queries=[vector_query],
        select=["id", "contractId", "vendorName", "content"],
        highlight_fields="vendorName",
        query_type="semantic",  # Enable semantic search
        semantic_configuration_name="default",  # You'll need to set this up
        search_fields=["vendorName", "clientName","contractTitle","sourceFileName","content"],
        top=3
    )

    print(f"\nQuery: {query}")
    
    # Get the first result from the iterator
    first_result = next(text_results, None)
    if first_result:
        print(f"Vendor: {first_result['vendorName']}")
    else:
        print("No results found.")
        
    print("\nText Search Results:")
    for result in text_results:
        print(f"Search.Score: {result['@search.score']}")
        print(f"Search.Reranker.Score: {result['@search.reranker_score']}")
        print(f"Search.Highlights: {result['@search.highlights']}")
        print(f"Vendor: {result['vendorName']}")
        print(f"Content Preview: {result['content'][:200]}...")
        print("---")

    print("\nHybrid Search Results:")
    for result in hybrid_results:
        print(f"Search.Score: {result['@search.score']}")
        print(f"Search.Reranker.Score: {result['@search.reranker_score']}")
        print(f"Search.Highlights: {result['@search.highlights']}")
        print(f"Vendor: {result['vendorName']}")
        print(f"Content Preview: {result['content'][:200]}...")
        print("---")

# Test queries
queries = [
    "Fabrikam",
    "IT service provider",
    "software development contractor",
    "consulting services",
    "What can you tell me about the vendor agreement between Fabrikam Services and Contoso Elite?"
]

for query in queries:
    compare_searches(query)


Query: Fabrikam
Vendor: Fabrikam Services

Text Search Results:
Search.Score: 0.5170628
Search.Reranker.Score: None
Search.Highlights: None
Vendor: Fabrikam Services
Content Preview: Amendment to Vendor Contractor Agreement Contract ID: 5004432-A1 Parent Contract ID: 5004432
This Amendment ("Amendment") to the Vendor Contractor Agreement (the "Agreement") is entered into as of 12/...
---

Hybrid Search Results:
Search.Score: 0.03306011110544205
Search.Reranker.Score: 2.5451736450195312
Search.Highlights: {'vendorName': ['<em>Fabrikam</em> Services']}
Vendor: Fabrikam Services
Content Preview: Vendor Contractor Agreement
Contract ID: 5004432
This Vendor Contractor Agreement ("Agreement") is entered into as of 12/7/2024 by and between:
. [Contoso Elite] (the "Company"), with its principal of...
---
Search.Score: 0.03306011110544205
Search.Reranker.Score: 2.4604554176330566
Search.Highlights: {'vendorName': ['<em>Fabrikam</em> Services']}
Vendor: Fabrikam Services
Content Preview: Amendm